In [ ]:
import pandas as pd

file_path = "./Book2.xlsx"
df = pd.read_excel(file_path)

json_data = df.to_json(orient="records", indent=4, force_ascii=False)

output_path = "output.json"  # Replace with your desired output file path
with open(output_path, "w", encoding="utf-8") as json_file:
    json_file.write(json_data)

print("JSON Output:\n", json_data)


In [7]:
import json

# Load JSON data
json_file = "./output.json"
with open(json_file, encoding="utf-8") as file:
    data = json.load(file)

# Extract relevant fields and remove duplicates
loi_vi_pham = set()
phuong_tien = set()
hinh_phat = set()
chi_tiet_loi = set()

for item in data:
    # print(item.get("Lỗi vi phạm", ""))
    loi_vi_pham.add((item.get("Lỗi vi phạm", "") or "").strip())
    phuong_tien.add((item.get("Phương tiện", "") or "").strip().lower().rstrip('.'))
    hinh_phat.add((item.get("Mức phạt", "") or "").strip().lower().rstrip('.'))
    chi_tiet_loi.add((item.get("Chi tiết lỗi", "") or "").strip().rstrip('.'))

# Remove null or empty entries
loi_vi_pham.discard("")
phuong_tien.discard("")
hinh_phat.discard("")
chi_tiet_loi.discard("")

In [8]:
# Create MySQL INSERT commands
def create_insert_commands(table_name, column_name, data_set):
    commands = []
    for value in data_set:
        commands.append(f"INSERT IGNORE INTO {table_name} ({column_name}) VALUES ('{value.replace("'", "''")}');")
    return commands

loi_vi_pham_commands = create_insert_commands("LoiViPham", "NoiDung", loi_vi_pham)
phuong_tien_commands = create_insert_commands("PhuongTien", "TenPhuongTien", phuong_tien)
hinh_phat_commands = create_insert_commands("HinhPhat", "NoiDung", hinh_phat)
chi_tiet_loi_commands = create_insert_commands("ChiTietLoi", "NoiDung", chi_tiet_loi)

# Combine all commands
all_commands = (
    loi_vi_pham_commands
    + phuong_tien_commands
    + hinh_phat_commands
    + chi_tiet_loi_commands
)

# Save to SQL file
with open("insert_commands.sql", "w", encoding="utf-8") as sql_file:
    for command in all_commands:
        sql_file.write(command + "\n")

# Print completion message
print("SQL insert commands saved to 'insert_commands.sql'")

SQL insert commands saved to 'insert_commands.sql'


In [1]:
import mysql.connector
from mysql.connector import Error
import json

In [3]:
# Step 1: Load JSON data
json_file = "./output.json"  # Replace with your JSON file path
with open(json_file, encoding="utf-8") as file:
    data = json.load(file)

# Step 2: Connect to MySQL
try:
    connection = mysql.connector.connect(
    host="localhost",  # Replace with your MySQL host
    user="root",       # Replace with your MySQL username
    password="Phanvylqd@112",  # Replace with your MySQL password
    database="luatgiaothong"  # Replace with your database name
    )
    cursor = connection.cursor()

    for item in data:
        # phuong_tien.add((item.get("Phương tiện", "") or "").strip().lower().rstrip('.'))
        # hinh_phat.add((item.get("Mức phạt", "") or "").strip().lower().rstrip('.'))

        loi_vi_pham = (item.get("Lỗi vi phạm", "") or "").strip() or None
        phuong_tien = (item.get("Phương tiện", "") or "").strip().lower().rstrip('.') or None
        chi_tiet_loi = (item.get("Chi tiết lỗi", "") or "").strip().rstrip('.') or None
        hinh_phat = (item.get("Mức phạt", "") or "").strip().lower().rstrip('.') or None
        dieu_khoan = (item.get("Căn cứ pháp lý", "") or "").strip() or ""
        ngay_ap_dung = (item.get("Ngày áp dụng", "") or "").strip() or None

        # Convert date format
        if ngay_ap_dung:
            day, month, year = ngay_ap_dung.split("/")
            ngay_ap_dung = f"{year}-{month}-{day}"

        # Get IDs for LoiViPham, PhuongTien, ChiTietLoi, and HinhPhat
        def get_id(table_name, column_name, value):
            if not value:
                return None
            cursor.execute(f"SELECT {table_name}ID FROM {table_name} WHERE {column_name} = %s LIMIT 1", (value,))
            result = cursor.fetchone()
            return result[0] if result else None

        loi_vi_pham_id = get_id("LoiViPham", "NoiDung", loi_vi_pham)
        phuong_tien_id = get_id("PhuongTien", "TenPhuongTien", phuong_tien)
        chi_tiet_loi_id = get_id("ChiTietLoi", "NoiDung", chi_tiet_loi)
        hinh_phat_id = get_id("HinhPhat", "NoiDung", hinh_phat)

        # Insert data into `Luat` table
        insert_luat_query = """
            INSERT INTO Luat (LoiViPhamID, PhuongTienID, ChiTietLoiID, HinhPhatID, DieuKhoan, NgayApDung)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_luat_query, (
            loi_vi_pham_id,
            phuong_tien_id,
            chi_tiet_loi_id,
            hinh_phat_id,
            dieu_khoan,
            ngay_ap_dung
        ))

    # Commit the changes
    connection.commit()
    print("Data successfully inserted into the `Luat` table!")

except Error as e:
    print("Error while connecting to MySQL:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

Data successfully inserted into the `Luat` table!
MySQL connection closed.
